# Analyzing Sea Level Rise in the Cloud with Coiled and Earthaccess

_This notebook was adapted from [this NASA Earthdata Cloud Cookbook example](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/tutorials/Sea_Level_Rise.html)_

## Get data files with `earthaccess`

In [ ]:
# Authenticate my machine with `earthaccess`
import earthaccess

earthaccess.login();

In [ ]:
%%time

# Retrieve data files for the dataset I'm interested in
granules = earthaccess.search_data(
    short_name="SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205",
    temporal=("2000", "2019"),
)

## Define processing function

In [ ]:
import coiled
import xarray as xr

@coiled.function(region="us-west-2")  # Same region as data
def process(granule, fs):
    results = []
    for file in granule.data_links("direct"):
        ds = xr.open_dataset(fs.open(file))
        ds = ds.sel(Latitude=slice(23, 50), Longitude=slice(270, 330))
        ds = ds.SLA.where((ds.SLA >= 0) & (ds.SLA < 10))
        results.append(ds)
    result = xr.concat(results, dim="Time")
    return result

## Process Granules

In [ ]:
fs = earthaccess.get_s3fs_session(results=granules)

### Process single file

In [ ]:
chunk = process(granules[0], fs=fs)
chunk.plot(x="Longitude", y="Latitude", figsize=(14, 6));

### Process all files in parallel

In [ ]:
%%time

chunks = process.map(granules, fs=fs)  # This runs on the cloud in parallel
ds = xr.concat(chunks, dim="Time")
ds.std("Time").plot(x="Longitude", y="Latitude", figsize=(14, 6));